In [ ]:
import os
import sys
import joblib
n_jobs = 8
import yaml

If you use yaml file to set parameters, use below cell as code.

import yaml
conf_file = "conf.yaml"
with open(conf_file, "r") as f:
    conf = yaml.safe_load(f)

If you don't use yaml file to set parameters, use below.


|  key  |  description  |
| :---: | :--- |
|use_devices | use gpu number. If you want to do distributed learning, write like "0,1,2" |
|model_dir |all outputs are written here. <br>e.g., trained model, inference results, etc..|
| train_x_dirs<br> valid_x_dirs<br> test_x_dirs | list of directories where input images exist.|
|  train_y_dirs<br> valid_y_dirs                   | list of directories where segmentation image exsit.<br> Each segmentation image name must be the same for corresponding input image.  |
|which_to_inference <br> which_to_visualise          | chosse from "train", "valid", "test".|
|output_activation                                   | chosse "softmax" or "sigmoid". <br>softmax means each pixcel is assigned to 1 label.<br>sigmoid means each pixel can assigned 1 or more labels.|
|batch_size | batch size|
|n_epochs   |nuber of epochs |
|image_size | [height, width] |
|optimizer  |"Adam" or "Nadam" or "SGD" |
|loss       |choose one .<br>"CE": cross entropy <br> "FL": focal loss <br>"GDL": generalized dice loss
|metrics| choose one. <br> IoU: Intersection over Union. Jaccard index. <br>F1score: F1 score. F-measure. Dice coefficient.|
|check_categorical_metrics| True, if you want to check metrics for each label.|
|use_tensorboard| if you want to use, True. If not, False.|


In [ ]:
conf = dict()

conf["model_dir"] = "path/to/output_dir"
conf["label_file_path"] = "path/to/label_list.csv"

# these are not needed for visualising.
# conf["train_x_dirs"] = ["path/to/train_x1", "path/to/train_x2"]
# conf["train_y_dirs"] = ["path/to/train_y1", "path/to/train_y2"]
# conf["valid_x_dirs"] = ["path/to/valid_x1", "path/to/valid_x2"]
# conf["valid_y_dirs"] = ["path/to/valid_y1", "path/to/valid_y2"]


conf["test_x_dirs"] = ["path/to/test_x1", "path/to/test_x2"] # if you don't have test data, enter empty list.([])
conf["which_to_visualise"] = ["valid", "test"]

# below lines are not needed in training phase.
# conf["which_to_inference"] = ["train", "valid", "test"]


# conf["output_activation"] = "softmax"

# conf["batch_size"] = 8
# conf["n_epochs"] = 1000
# conf["image_size"] = [512, 512]
# conf["optimzer"] = "Adam"
# conf["loss"] = "FL"
# conf["use_tensorboard"] = False


In [ ]:
model_dir = conf["model_dir"]
label_file_path = conf["label_file_path"]
test_x_dirs = conf["test_x_dirs"]

which_to_visualise = conf["which_to_visualise"]

import modules

In [ ]:
from deeplab_v3plus_tfkeras.label import Label
from deeplab_v3plus_tfkeras.data_utils import load_inference_results
from deeplab_v3plus_tfkeras.vis_utils import convert_y_to_image_array
from deeplab_v3plus_tfkeras.vis_utils import visualise_true_pred
from deeplab_v3plus_tfkeras.vis_utils import visualise_pred

some settings

In [ ]:
fig_dir = os.path.join(model_dir, "figure")
label = Label(label_file_path)

visualise inference results of training dataset.

In [ ]:
if "train" in which_to_visualise:
    fig_out_dir = os.path.join(fig_dir, "train")
    os.makedirs(fig_out_dir, exist_ok=True)

    fpath = os.path.join(model_dir, "train_inference.h5")
    x, y, pred, last_activation = load_inference_results(fpath)

    y_pred = convert_y_to_image_array(pred,
                                      label,
                                      threshold=0.5,
                                      activation=last_activation)
    y_true = convert_y_to_image_array(y,
                                      label,
                                      activation=last_activation)

    joblib.Parallel(
        n_jobs=n_jobs,
        verbose=10,
        backend="threading")(joblib.delayed(visualise_true_pred)(
            i,
            x,
            y_true,
            y_pred,
            fig_out_dir,
            last_activation,
            label) for i in range(x.shape[0]))

visualise inference results of validation dataset.

In [ ]:
if "valid" in which_to_visualise:
    fig_out_dir = os.path.join(fig_dir, "valid")
    os.makedirs(fig_out_dir, exist_ok=True)

    fpath = os.path.join(model_dir, "valid_inference.h5")
    x, y, pred, last_activation = load_inference_results(fpath)

    y_pred = convert_y_to_image_array(pred,
                                      label,
                                      threshold=0.5,
                                      activation=last_activation)
    y_true = convert_y_to_image_array(y,
                                      label,
                                      activation=last_activation)

    joblib.Parallel(
        n_jobs=n_jobs,
        verbose=10,
        backend="threading")(joblib.delayed(visualise_true_pred)(
            i,
            x,
            y_true,
            y_pred,
            fig_out_dir,
            last_activation,
            label) for i in range(x.shape[0]))


visualise inference results of test dataset.

In [ ]:
if "test" in which_to_visualise:
    for test_data_dir in test_x_dirs:
        print(test_data_dir)
        test_name = test_data_dir.split(os.sep)[-1]
        fig_out_dir = os.path.join(fig_dir, "test_" + test_name)
        os.makedirs(fig_out_dir, exist_ok=True)

        fpath = os.path.join(model_dir, "test_" + test_name + "_inference.h5")
        x, _, pred, last_activation = load_inference_results(fpath)

        y_pred = convert_y_to_image_array(pred,
                                          label,
                                          threshold=0.5,
                                          activation=last_activation)

        joblib.Parallel(
            n_jobs=n_jobs,
            verbose=10,
            backend="threading")(joblib.delayed(visualise_pred)(
                i,
                x,
                y_pred,
                fig_out_dir,
                last_activation,
                label) for i in range(len(x)))
